In [1]:
import pandas as pd # Pandas is an open source library providing data structures and data analysis tools for Python
import re #regex
from functools import reduce 
import numpy as np
from math import isnan
import matplotlib.pyplot as plt
import urllib.request
import json
from datetime import date
from datetime import datetime

#### TODAY'S DATE

In [2]:
def containQuoteMarks(string):
    if string.startswith("\"") and string.endswith("\""):
        return True
today = 'jan'
beta_file_loc =  "C:/Users/huixin/OneDrive - UC San Diego/menoplan/pilot_phase/merged_analysis/"
pilot_file_loc = "C:/Users/huixin/OneDrive - UC San Diego/menoplan/trial_phase/"
cleaned = pd.read_csv(pilot_file_loc + "cleaned_data/p2/cleaned" + today  + "_p2.csv")
#screening = pd.read_csv(pilot_file_loc +"qualtrics_data/" + today+ "/"+ today+ "_p1.csv")
#screening=screening.iloc[2:] #drop first two rows
mo='jan'
allDays = [str(d)+mo for d in range(12,24)]
li=[]
for d in allDays: #survey 1 only
    df = pd.read_csv(pilot_file_loc +"qualtrics_data/" + d + "/"+ d + "_p1.csv")
    df = df.iloc[2:] 
    li.append(df)
p1 = pd.concat(li, axis=0, ignore_index=True)
screening= p1
screening=screening.add_suffix('_p1')
screening=screening.rename(columns={#'IPAddress_p1':'IPAddress_p2', 
                                   'Duration (in seconds)_p1': 'durationMins_p1'})
#Merge using Email Address
cleaned['EmailMerge'] = cleaned['RecipientEmail_p2']
screening['EmailMerge'] = screening['Email_p1']
allD = pd.merge(screening, cleaned, on=['EmailMerge'], how='inner')
allD['durationMins_p1']=(pd.to_numeric(allD['durationMins_p1'])/60)
allD['Q_RecaptchaScore_p1']=pd.to_numeric(allD['Q_RecaptchaScore_p1'])
allD['Q_RelevantIDFraudScore_p1']=pd.to_numeric(allD['Q_RelevantIDFraudScore_p1'])
allD['cityName_p2']=allD['cityName_p2'].str.replace('city =', '') #remove string in front
allD['cityName_p1']=allD['cityName_p1'].str.replace('city =', '')

#THIS CONTAINS FRAUDULENT ENTRIES
allD  = allD[allD["I-Information_4_p2"].str.contains("Hui Xin")==False]#drop all rows that contain hui xin 
allD=allD.drop_duplicates()# delete any rows that are the same
allD['IPFreq']=allD['IPAddress_p2'].map(allD['IPAddress_p2'].value_counts())#count frequency of IP addresses

print(len(allD))
runChecks = allD

737


In [3]:
conDf = runChecks
print(len(conDf))
def getLsLen(string):
    li = list(string.split(","))
    return len(li)

##### create 'selectAll' col that checks if participant selects all options on a multi-choice question
conDf['selectAll']=conDf["I-Peri_Meno_Experie_p2"].apply(lambda x: (getLsLen(str(x))))
#calculate percentage of NAs for each condition
importantQCols = conDf.iloc[:, 48:-24].columns
conDf['percentNA_p2'] = conDf[importantQCols].isna().sum(axis = 1)/len(conDf[importantQCols].columns)
#check if street address has quotation marks around
conDf['QuoteMarks']= conDf['I-Information_5_p2'].apply(lambda x: containQuoteMarks(str(x)))
#check if they picked 
conDf['passATT']=(conDf['I-ATT-CHECK1_p2']==1) | (conDf['I-ATT-CHECK2_p2']!=5) 

cond1=conDf["I-Peri_Meno_Experie_p2"].str.contains(',11') #select other options even though selected 'I refuse to answer'

#cond2=conDf["I-ATT-CHECK1_p2"] != 1 #Did not select 'Strongly Agree'
#cond3=conDf["I-ATT-CHECK2_p2"] != 5 #Did not select 'Strongly Disagree'

cond5=conDf['passATT'] != True
cond6 = ~conDf['Randomization_2_p2'].isin(list(np.arange(61, 82, 1))) #age out of range
cond7 = ~conDf['I-Gender_p2'].isin([1,4,5]) #not female, transgender male, or non binary
conDf['IPAddDup_p2']=conDf.duplicated(['IPAddress_p2']) #IP address duplicated

cond8= conDf['IPAddDup_p2'] ==True #IP address duplicated
cond9=conDf['durationMins_p2'] <=2 #finished survey in less than 2 mins
#cond10 = conDf['zipcodeX_p2'] != True #entered zipcode's state does not match entered state
cond4=conDf['RecipientEmail_p2'].str.isupper() #all Caps, was the email variable removed?
cond11 = conDf['selectAll'] == 11
cond12 = conDf['durationMins_p2'] < 30

recaptcha = conDf['Q_RecaptchaScore_p2'] < 0.5 
cond13=conDf['Q_RelevantIDFraudScore_p2'] >=30
cond14 = conDf['Q_RelevantIDDuplicate_p2']==True

recaptcha_p1 = conDf['Q_RecaptchaScore_p1'] < 0.5
cBot=conDf['Q_RelevantIDFraudScore_p1'] >= 30
cDuplicateBool=conDf['Q_RelevantIDDuplicate_p1']==True
allD['Fraud'] = False
numFraudPerCheck = []
conNoFraud = allD

for x in [recaptcha, recaptcha_p1 , cBot,cDuplicateBool,cond1, cond5 , cond4 , 
          cond6, cond7, cond8,cond9, cond11, cond12
         , cond13, cond14]:
    if (x.any()==True):
        allD.loc[allD[x].index, 'Fraud'] = True
        numFraudPerCheck.append({x.name: len(allD[x])})
nData=len(allD)
allD.sort_values(by=['IPAddress_p2']).to_csv(pilot_file_loc+"cleaned_data/allData"+ today+ ".csv", index=False)

737


In [4]:
percentFraudEachVar = pd.DataFrame.from_dict(numFraudPerCheck).sum()/len(allD)
print("Percent of surveys that failed the following checks among " + str(len(allD)) + " surveys")
print(round(percentFraudEachVar, 3))

Percent of surveys that failed the following checks among 737 surveys
Q_RecaptchaScore_p2          0.005
Q_RecaptchaScore_p1          0.005
Q_RelevantIDFraudScore_p1    0.052
passATT                      0.153
Randomization_2_p2           0.008
I-Gender_p2                  0.486
IPAddDup_p2                  0.005
durationMins_p2              0.278
Q_RelevantIDFraudScore_p2    0.054
Q_RelevantIDDuplicate_p2     0.012
dtype: float64


In [5]:
#Examples of duplicated IPAddresses in survey 2 due to fraud
#conDf[conDf.duplicated(['IPAddress_p2']) == True].sort_values(by=['StartDate_p1','IPAddress_p2'])

### Remove all Fraudulent Entries

In [6]:
totalPercentFraud = []
for x in [recaptcha, recaptcha_p1 , cBot,cDuplicateBool,cond1, cond5 , cond4 , 
          cond6, cond7, cond8,cond9, cond11, cond12
         , cond13, cond14]:
    if (x.any()==True):
       totalPercentFraud.append({x.name: len(conNoFraud[x])})
       #conNoFraud[]
       conNoFraud.drop(conNoFraud[x].index, inplace = True) 
#totalPercentFraud
len(conDf)

C:\Users\huixin\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  
C:\Users\huixin\anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


218

In [7]:
print('Percent of surveys that are fraudulent, out of ' + str((nData)) + ' surveys')
pd.DataFrame.from_dict(totalPercentFraud).sum().sum()/(nData)


Percent of surveys that are fraudulent, out of 737 surveys


0.7042062415196744

In [8]:
#CONTAINS NO FRAUDULENT USER DATA, WITH PERSONAL INFORMATION
conNoFraud.to_csv(pilot_file_loc + "cleaned_data/noFraud/noFraud"+ today+".csv", index=False)

### Remove all Fraudulent Enries and Personal Information

In [9]:
allDRemoveId = conNoFraud
allDRemoveId = (allDRemoveId.drop(['RecipientLastName_p1','RecipientFirstName_p1','RecipientEmail_p1',
 'ExternalReference_p1',
 'LocationLatitude_p1','LocationLongitude_p1', 'IPAddress_p2' ,'LocationLatitude_p2', 'LocationLongitude_p2','RecipientEmail_p2',
 'RecipientFirstName_p2','RecipientLastName_p2', 'I-Phone_p2', 'I-Phonex2_p2', 'Email_p1','Email2_p1', 'Email_p2', 'I-Information_11_p2',
 'I-Information_16_p2', 'I-Information_4_p2','I-Information_5_p2','I-Information_7_p2','I-Information_8_p2','I-Information_9_p2'], axis=1))

###  FRAUDULENT ENTRIES AND PERSONAL IDENTIFIERS REMOVED - ANON DATA WITH NO FRAUDSTERS
allDRemoveId.to_csv(pilot_file_loc + "anonymized_data/noFraudAnon"+ today+".csv", index=False)

In [10]:
list(allD.columns)

['StartDate_p1',
 'EndDate_p1',
 'Status_p1',
 'IPAddress_p1',
 'Progress_p1',
 'durationMins_p1',
 'Finished_p1',
 'RecordedDate_p1',
 'ResponseId_p1',
 'RecipientLastName_p1',
 'RecipientFirstName_p1',
 'RecipientEmail_p1',
 'ExternalReference_p1',
 'LocationLatitude_p1',
 'LocationLongitude_p1',
 'DistributionChannel_p1',
 'UserLanguage_p1',
 'Q_RecaptchaScore_p1',
 'Q_RelevantIDDuplicate_p1',
 'Q_RelevantIDDuplicateScore_p1',
 'Q_RelevantIDFraudScore_p1',
 'Q_RelevantIDLastStartDate_p1',
 'Q15_p1',
 'IS1_p1',
 'SQ1_p1',
 'SQ2_p1',
 'SQ3_p1',
 'SQ4_p1',
 'BrowserInfo_Browser_p1',
 'BrowserInfo_Version_p1',
 'BrowserInfo_Operating System_p1',
 'BrowserInfo_Resolution_p1',
 'Email_p1',
 'Email2_p1',
 'QCity_ExportTag_p1',
 'QState_ExportTag_p1',
 'Referer_p1',
 'cityName_p1',
 'Q_URL_p1',
 'Q_BallotBoxStuffing_p1',
 'Q17_Browser_p1',
 'Q17_Version_p1',
 'Q17_Operating System_p1',
 'Q17_Resolution_p1',
 'countryName_p1',
 'EmailMerge',
 'Cond_p2',
 'DistributionChannel_p2',
 'Educ_p2',